In [157]:
from theano.sandbox import cuda

In [158]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [159]:
model_path = 'data/models/'

In [160]:
from pandas import read_csv

In [161]:
rows = read_csv("data/tc_articles.csv", skiprows=1)

In [ ]:
articles = rows.values[np.random.choice(range(0, len(rows.values)), 5000)]

In [ ]:
articles = articles.flatten()

In [352]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [350]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [354]:
idx2word = {v: k for k, v in idx.iteritems()}

In [363]:
articles = [' '.join([idx2word[word] for word in review]) for review in x_train]

In [364]:
len(articles)

25000

In [365]:
articles[:5]

["bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't",
 "homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter mo

In [366]:
idx = {}
article_words = [name.decode('unicode_escape').encode('ascii','ignore').split(" ") for name in articles]
words = [item for sublist in article_words for item in sublist]

In [367]:
for i in range(0, len(words)):
    if not words[i] in idx:
        idx[words[i]] = 1
    else:
        idx[words[i]] += 1;

In [368]:
idx_arr = sorted(idx, key=idx.get, reverse=True)

In [369]:
vocab_size = 5000

In [370]:
idx2word = {idx_arr.index(v): v for v in idx_arr}

In [371]:
word2idx = {v: k for k, v in idx2word.iteritems()}

In [377]:
import pickle
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))


In [373]:
vecs, words, wordidx = load_vectors('data/6B.50d')

In [378]:
import re
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word) and word in wordidx:
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [379]:
emb = create_emb()

In [380]:
from keras.layers import *
from keras.layers.normalization import *

In [381]:
n_fact = vecs.shape[1]

In [390]:
word_length = min(10, int(np.floor(np.mean([len(wds) for wds in article_words]))))

In [383]:
batch_size = 64

In [384]:
n_hidden = 256

In [387]:
list(xrange(0, max(len(article_words[0])-1, word_length-1)))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [391]:
c_in_dat = [[np.clip(word2idx[article_words[articleidx][widx]], 0, vocab_size-1) for widx in range(0, min(len(article_words[articleidx])-1, word_length-1))]
            for articleidx in range(len(article_words))]

In [392]:
c_out_dat = [[np.clip(word2idx[article_words[articleidx][widx]], 0, vocab_size-1) for widx in range(1, min(len(article_words[articleidx]), word_length))]
            for articleidx in range(len(article_words))]

In [393]:
x_rnn = sequence.pad_sequences(c_in_dat, maxlen=word_length, value=0)
ys = sequence.pad_sequences(c_out_dat, maxlen=word_length, value=0)

In [394]:
x_rnn = x_rnn.clip(0, vocab_size-1)
y_rnn = y_rnn.clip(0, vocab_size-1)

In [395]:
x_rnn.shape

(25000, 10)

In [396]:
y_rnn = np.expand_dims(np.array(ys), -1)

In [397]:
x_rnn.shape, y_rnn.shape

((25000, 10), (25000, 10, 1))

In [398]:
emb.shape

(5000, 50)

In [399]:
model=Sequential([
        Embedding(vocab_size, n_fact, input_length=word_length, 
                  weights=[emb],
                  trainable=False,
                  batch_input_shape=(batch_size, word_length)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])


In [400]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=1e-15))

In [401]:
mx = len(x_rnn)//batch_size*batch_size

In [402]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=batch_size, nb_epoch=4, shuffle=False)

Epoch 1/4
24960/24960 [==============================] - 26s - loss: 8.5185    
Epoch 2/4
24960/24960 [==============================] - 25s - loss: 8.5185    
Epoch 3/4
24960/24960 [==============================] - 25s - loss: 8.5185    
Epoch 4/4
24960/24960 [==============================] - 25s - loss: 8.5185    


In [408]:
def get_nexts_keras(inp):
    idxs = [word2idx[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [idx2word[np.argmax(o)] for o in p]

In [410]:
get_nexts_keras(['', 'that', '', '', '', '', '', '', '', ''])

ValueError: dimension mismatch in args to gemm (64,256)x(256,256)->(1,256)
Apply node that caused the error: GpuGemm{no_inplace}(GpuSubtensor{::, int64::}.0, TensorConstant{0.20000000298}, <CudaNdarrayType(float32, matrix)>, lstm_7_U_o_copy[cuda], TensorConstant{0.20000000298})
Toposort index: 5
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(float32, scalar), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), TensorType(float32, scalar)]
Inputs shapes: [(1, 256), (), (64, 256), (256, 256), ()]
Inputs strides: [(0, 1), (), (256, 1), (256, 1), ()]
Inputs values: ['not shown', array(0.20000000298023224, dtype=float32), 'not shown', 'not shown', array(0.20000000298023224, dtype=float32)]
Outputs clients: [[GpuElemwise{Composite{(clip((i0 + i1), i2, i3) * tanh(i4))},no_inplace}(CudaNdarrayConstant{[[ 0.5]]}, GpuGemm{no_inplace}.0, CudaNdarrayConstant{[[ 0.]]}, CudaNdarrayConstant{[[ 1.]]}, GpuElemwise{Composite{((clip((i0 + i1), i2, i3) * i4) + (clip((i0 + i5),